# Reconhecimento de Sinais em LIBRAS com Landmarks de Mãos

## 1. Instalação e Imports

Instale as dependências (execute apenas se necessário)
> !pip install opencv-python mediapipe pandas numpy tensorflow pyarrow scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from src.dataset_utils import load_metadata, load_landmarks
from src.model import build_transformer_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from model.src.preprocessing import preprocess_landmarks
import matplotlib.pyplot as plt

## 2. Carregar Metadados

In [ ]:
MODEL_PATH = "../saved_models/model_libras.h5"
ENCODER_PATH = "../saved_models/label_encoder.pkl"

In [ ]:
metadata = load_metadata('../data/train.csv')
print(metadata.head())

## 3. Codificar Labels

In [ ]:
label_encoder = LabelEncoder()
metadata['label_encoded'] = label_encoder.fit_transform(metadata['label'])
num_classes = len(label_encoder.classes_)
print("Classes:", label_encoder.classes_)

## 4. Carregar Landmarks e Preparar Dados

In [ ]:
MAX_FRAMES = 30  # Ajuste conforme a duração média dos seus vídeos

X = []
y = []

for _, row in metadata.iterrows():
    df = load_landmarks(row['path'])
    df = df.fillna(0)
    arr = df.drop(columns=['frame'], errors='ignore').values
    # Padronizar para MAX_FRAMES frames
    if len(arr) >= MAX_FRAMES:
        arr = arr[:MAX_FRAMES]
    else:
        arr = np.pad(arr, ((0, MAX_FRAMES - len(arr)), (0, 0)))
    X.append(arr)
    y.append(row['label_encoded'])

X = np.stack(X)
y = to_categorical(y, num_classes=num_classes)

print("Shape X:", X.shape)
print("Shape y:", y.shape)

In [ ]:
from tensorflow.keras.models import load_model
import pickle

model = load_model('saved_models/model_libras.h5')
with open('saved_models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Pré-processar um novo arquivo de landmarks
from model.src.preprocessing import preprocess_landmarks
df = pd.read_parquet('caminho/para/novo_landmark.parquet')
features = preprocess_landmarks(df)
pred = model.predict(features.reshape(1, -1))
label = label_encoder.inverse_transform([pred.argmax()])[0]
print("Predição:", label)

## 5. Dividir em Treino e Validação

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Treino:", X_train.shape, "Validação:", X_val.shape)

## 6. Construir e Compilar o Modelo

In [ ]:
input_shape = X_train.shape[1:]  # (frames, features)
model = build_transformer_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 7. Treinar o Modelo

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=8
)

## 8. Avaliar o Modelo

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Acurácia: {acc:.2%}")

In [ ]:
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.show()

## 9. Salvar o Modelo e o Encoder

In [ ]:
model.save('../saved_models/model_libras.h5')
import pickle
with open('../saved_models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

## 10. Inferência em Novos Vídeos

In [ ]:
def predict_landmarks(landmarks_array):
    arr = landmarks_array
    if len(arr) >= MAX_FRAMES:
        arr = arr[:MAX_FRAMES]
    else:
        arr = np.pad(arr, ((0, MAX_FRAMES - len(arr)), (0, 0)))
    arr = np.expand_dims(arr, axis=0)
    pred = model.predict(arr)
    label_idx = np.argmax(pred)
    return label_encoder.inverse_transform([label_idx])[0]

# Exemplo de uso:

In [ ]:
# df_new = load_landmarks('CAMINHO/DO/SEU/ARQUIVO.parquet')
# print(predict_landmarks(df_new.drop(columns=['frame'], errors='ignore').values))